In [32]:
import yfinance as yf
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import talib as ta
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load gold historical data from Yahoo Finance
data = yf.download('^XAU', start='2019-01-01', end='2024-01-01') # โหลดข้อมูลมาแล้วระบุวันเริ่มกับวันสุดท้าย

[*********************100%***********************]  1 of 1 completed


In [3]:
data['Return'] = data['Close'].pct_change() # ทำค่าผลต่างเป็นเปอร์เซ็นจากวันปัจจุบันและวันก่อนหน้า return ก็คือผลตอบแทนที่ดได้จากราคาปิดวันนี้เทียบกับเมื่อวาน

# Add more technical indicators
# หา Simple Moving Average ช่วง 10 และ 50 วัน
data['SMA_15'] = ta.SMA(data['Close'].to_numpy().flatten(), timeperiod=15)
data['SMA_30'] = ta.SMA(data['Close'].to_numpy().flatten(), timeperiod=30)
data['SMA_60'] = ta.SMA(data['Close'].to_numpy().flatten(), timeperiod=60)

# หา Relative Strength Index ในช่วง 14 วัน
# หาก RSI ต่ำกว่า 30% จะเรียกว่าภาวะ “ขายมากเกินไป” (Oversold)
# หาก RSI สูงกว่า 70% จะเรียกว่าภาวะ “ซื้อมากเกินไป” (Overbought)
data['RSI'] = ta.RSI(data['Close'].to_numpy().flatten(), timeperiod=14)

# หา Moving Average Convergence Divergence ตัว MACD หาได้จากความต่างของ EMA (Exponential Moving Average) สองเส้นหรือ EMA12 - EMA26
# เอาไว้ดูคู่กับ signal line เพื่อประกอบการตัดสินใจจังหวะซื้อหรือขาย
data['MACD'], data['MACD_Signal'], _ = ta.MACD(data['Close'].to_numpy().flatten(), fastperiod=12, slowperiod=26, signalperiod=9)

# Bollinger Bands กรอบราคาที่สามารเหวี่ยงไปได้ทั้งกรอบบน กรอบล่าง
data['BB_upper'], data['BB_middle'], data['BB_lower'] = ta.BBANDS(data['Close'].to_numpy().flatten(), timeperiod=20)

# หา momentum ซึ่งเป็นอัตราเร่งของราคา
data['Momentum'] = ta.MOM(data['Close'].to_numpy().flatten(), timeperiod=10)

# หาความผันผวนของราคาปิดเมื่อเทียบกับราคาสูงสุดและต่ำสุด
data['Volatility'] = ta.ATR(data['High'].to_numpy().flatten(), data['Low'].to_numpy().flatten(), data['Close'].to_numpy().flatten(), timeperiod=14)  # Added Volatility

# หาค่า Stochastic ออกมาเป็นเปอร์เซ็นเพื่อดูพฤติกรรมการซื้อขาย เพื่อหาจุดที่จะเก็งกำไร ส่วนใหญ่ทำในระยะสั้น
data['Stochastic'] = ta.STOCH(data['High'].to_numpy().flatten(), data['Low'].to_numpy().flatten(), data['Close'].to_numpy().flatten(), fastk_period=14, slowk_period=3)[0]  # Added Stochastic

In [4]:
# ทำ label เป็น buy, sell
data['Target'] = np.where(data['Return'].shift(-1) > 0, 1, 0)
data.head()

Price,Adj Close,Close,High,Low,Open,Volume,Return,SMA_15,SMA_30,SMA_60,RSI,MACD,MACD_Signal,BB_upper,BB_middle,BB_lower,Momentum,Volatility,Stochastic,Target
Ticker,^XAU,^XAU,^XAU,^XAU,^XAU,^XAU,,,,,,,,,,,,,,
Date,,,,,,,,,,,,,,,,,,,,
2019-01-02 00:00:00+00:00,70.940002,70.940002,71.790001,69.860001,70.440002,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2019-01-03 00:00:00+00:00,72.169998,72.169998,72.389999,70.860001,71.470001,0,0.017339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-01-04 00:00:00+00:00,71.870003,71.870003,72.230003,70.570000,71.419998,0,-0.004157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-01-07 00:00:00+00:00,71.199997,71.199997,72.459999,70.980003,72.339996,0,-0.009322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2019-01-08 00:00:00+00:00,71.470001,71.470001,71.669998,70.230003,70.769997,0,0.003792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
# Drop NaN values after adding new features
data.dropna(inplace=True)

def split_data(data, data_range, train_ratio=0.8):
    split_range = int(len(data) * train_ratio)

    features = [
        'Return', 'SMA_15', 'SMA_30', 'SMA_60', 'RSI', 'MACD', 'MACD_Signal',
        'BB_upper', 'BB_middle', 'BB_lower', 'Momentum', 'Volatility', 'Stochastic'
    ]

    pre_x_train = data.iloc[:split_range].loc[:,features]
    pre_y_train = data.iloc[:split_range].loc[:,["Target"]]
    pre_x_test = data.iloc[split_range:].loc[:,features]
    pre_y_test = data.iloc[split_range:].loc[:,["Target"]]

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    for i in range(len(pre_x_train)-data_range):
        # x_tmp = np.transpose(pre_x_train.iloc[i:i+data_range].to_numpy())
        x_tmp = pre_x_train.iloc[i:i+data_range].to_numpy()
        y_tmp = pre_y_train.iloc[i+data_range-1].to_numpy()
        x_train.append(x_tmp)
        y_train.append(y_tmp)

    for i in range(len(pre_x_test)-data_range):
        # x_tmp = np.transpose(pre_x_test.iloc[i:i+data_range].to_numpy())
        x_tmp = pre_x_test.iloc[i:i+data_range].to_numpy()
        y_tmp = pre_y_test.iloc[i+data_range-1].to_numpy()
        x_test.append(x_tmp)
        y_test.append(y_tmp)

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    return x_train, y_train, x_test, y_test

In [ ]:
x_train, y_train, x_test, y_test = split_data(data, data_range=30)

(929, 30, 13) (929, 1) (210, 30, 13) (210, 1)
(929, 28, 16)
(929, 10, 13)


d:\co_op\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# draf model

In [ ]:
def create_cnn1d_gru_model(input_shape, gru_units=64):
    model = models.Sequential()
    
    # CNN Layers (Convolutional Layer + MaxPooling)
    model.add(layers.Conv1D(32, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    
    # model.add(layers.Conv1D(64, 3, activation='relu'))
    # model.add(layers.MaxPooling1D(2))
    
    # model.add(layers.Conv1D(128, 3, activation='relu'))
    # model.add(layers.MaxPooling1D(2))
    
    # Flatten the output from CNN layers to be input to GRU
    model.add(layers.Flatten())
    
    # GRU Layer
    model.add(layers.Reshape((-1, 32)))  # Reshape for GRU, where -1 is sequence length, 128 is feature dimension
    model.add(layers.GRU(gru_units, return_sequences=False))  # GRU Layer
    
    # Fully Connected Layer (Dense Layer)
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Output layer (10 classes, change as needed)

    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss=losses.SparseCategoricalCrossentropy())
    
    return model

my_model = create_cnn1d_gru_model(x_train.shape[1:])

d:\co_op\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# my_model.summary()
history = my_model.fit(x_train, y_train, batch_size=100, epochs=10)
history.history

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6866
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6883
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6868
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6863
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6853
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6823
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6838
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6843
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6821
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6836


{'loss': [0.6869746446609497,
  0.6866706013679504,
  0.6863816976547241,
  0.6857356429100037,
  0.6856086254119873,
  0.685441255569458,
  0.6853477358818054,
  0.6848428845405579,
  0.683872401714325,
  0.6840329170227051]}

gpt ชุด 1

In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import backtrader as bt # อันนี้น่าสนใจในการทำ backtest

# Step 1: ดึงข้อมูลจาก Yahoo Finance
data = yf.download('BTC-USD', start='2023-11-20', end='2024-11-20', progress=False)

# Step 2: สร้าง Feature สำหรับ Machine Learning
data['price_roc'] = data['Close'].pct_change() * 100  # Rate of Change ของราคา
data['volume_roc'] = data['Volume'].pct_change() * 100  # Rate of Change ของ volume
data = data.dropna()  # ลบค่า NaN ที่เกิดจากการคำนวณ

# Step 3: กำหนด Target (1 = long, -1 = short)
data['target'] = 0
data.loc[data['price_roc'] > 0, 'target'] = 1  # long หากราคาเพิ่มขึ้น
data.loc[data['price_roc'] < 0, 'target'] = -1  # short หากราคาลดลง

# Step 4: การเตรียมข้อมูลสำหรับการฝึก Machine Learning
X = data[['price_roc', 'volume_roc']]  # Features
y = data['target']  # Target

# แบ่งข้อมูลเป็น train/test (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# สร้างและฝึกโมเดล Machine Learning
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ทำนายผลใน test set
y_pred = model.predict(X_test)

# แสดงผลการทำนาย
print(classification_report(y_test, y_pred))

# Step 5: สร้าง Strategy สำหรับ Backtrader
class MLStrategy(bt.Strategy):
    # กำหนดค่า parameter สำหรับการเข้าสู่ตลาด
    def __init__(self):
        self.model = model
        self.data_close = self.datas[0].close
        self.data_volume = self.datas[0].volume
        self.X = pd.DataFrame({
            'price_roc': self.data_close.pct_change() * 100,
            'volume_roc': self.data_volume.pct_change() * 100
        })
        self.X = self.X.fillna(0)  # แทนที่ NaN ด้วย 0 เพื่อให้ไม่เกิดข้อผิดพลาดในการทำนาย
        self.y_pred = self.model.predict(self.X.values)  # ทำนายคำสั่ง Long/Short
    
    def next(self):
        if self.y_pred[-1] == 1:  # หากทำนาย long
            if not self.position:  # ถ้ายังไม่ได้เปิด position
                self.buy()
        elif self.y_pred[-1] == -1:  # หากทำนาย short
            if not self.position:  # ถ้ายังไม่ได้เปิด position
                self.sell()

# Step 6: การทำ Backtest โดยใช้ Backtrader
cerebro = bt.Cerebro()

# แปลงข้อมูลเป็น backtrader feed
data_bt = bt.feeds.PandasData(dataname=data)

# เพิ่มข้อมูลลงใน cerebro
cerebro.adddata(data_bt)

# เพิ่ม strategy ที่สร้างขึ้น
cerebro.addstrategy(MLStrategy)

# ตั้งค่าขนาดการลงทุนเริ่มต้น
cerebro.broker.set_cash(10000)

# ตั้งค่าค่าคอมมิชชั่น (เช่น 0.1% ต่อการเทรด)
cerebro.broker.set_commission(commission=0.001)

# รัน backtest
cerebro.run()

# แสดงกราฟผลลัพธ์
cerebro.plot()

gpt ชุด 2

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ตัวอย่างการสร้างโมเดล CNN + GRU

# สร้างโมเดล CNN + GRU
def create_cnn_gru_model(input_shape, gru_units=64):
    model = models.Sequential()
    
    # CNN Layers (Convolutional Layer + MaxPooling)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Flatten the output from CNN layers to be input to GRU
    model.add(layers.Flatten())
    
    # GRU Layer
    model.add(layers.Reshape((-1, 128)))  # Reshape for GRU, where -1 is sequence length, 128 is feature dimension
    model.add(layers.GRU(gru_units, return_sequences=False))  # GRU Layer
    
    # Fully Connected Layer (Dense Layer)
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer (10 classes, change as needed)
    
    return model

# สร้างโมเดล CNN + GRU
input_shape = (64, 64, 3)  # ขนาดของภาพ (64x64 พิกเซล, 3 แชนแนล RGB)
model = create_cnn_gru_model(input_shape)

# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# แสดงสรุปของโมเดล
model.summary()

# ตัวอย่างการฝึกโมเดล
# สมมุติว่าเรามีข้อมูล train_X และ train_Y เป็นข้อมูลที่ประกอบด้วยลำดับของภาพ
train_X = np.random.rand(100, 10, 64, 64, 3)  # 100 ตัวอย่าง, 10 ภาพในแต่ละลำดับ
train_Y = np.random.randint(0, 10, (100,))  # 100 ป้ายกำกับ (10 คลาส)

# การฝึกโมเดล
model.fit(train_X, train_Y, epochs=10, batch_size=16)
